In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://172.16.18.84:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1701128295444)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._


In [22]:
val spark = SparkSession
    .builder
    .appName("ch4")
    .enableHiveSupport()
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7d477eb9


In [5]:
val csv_file = "./data/departuredelays.csv"

csv_file: String = ./data/departuredelays.csv


In [6]:
val df = spark.read.format("csv")
    .option("inferSchema", true)
    .option("header", true)
    .load(csv_file)

df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [7]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [9]:
spark.sql("""
select distance, origin, destination
from us_delay_flights_tbl
where distance > 1000
order by distance desc
""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [10]:
spark.sql("""SELECT date, delay, origin, destination
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [12]:
spark.sql(
"""
select
    delay, origin, destination,
    case
        when delay > 360 then 'very long delays'
        when delay > 120 and delay < 360 then 'long delays'
        when delay > 60 and delay < 120 then 'short delays'
        when delay > 0 and delay < 60 then 'tolerable delays'
        when delay == 0 then 'no delays'
        else 'early'
    end as flight_delays
from us_delay_flights_tbl
order by origin, delay desc
"""
).show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|flight_delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  long delays|
|  305|   ABE|        ATL|  long delays|
|  275|   ABE|        ATL|  long delays|
|  257|   ABE|        ATL|  long delays|
|  247|   ABE|        ATL|  long delays|
|  247|   ABE|        DTW|  long delays|
|  219|   ABE|        ORD|  long delays|
|  211|   ABE|        ATL|  long delays|
|  197|   ABE|        DTW|  long delays|
|  192|   ABE|        ORD|  long delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [13]:
spark.sql("CREATE DATABASE learn_spark_db")

res5: org.apache.spark.sql.DataFrame = []


In [14]:
spark.sql("USE learn_spark_db")

res6: org.apache.spark.sql.DataFrame = []


In [28]:
spark.conf.get("spark.sql.warehouse.dir")

res13: String = file:/Users/jiashu/Documents/StudyNotes/spark/examples/spark-warehouse


In [31]:
spark.conf.get("spark.sql.catalogImplementation")

res16: String = in-memory


In [32]:
spark.sql("""
create table us_delay_flights_tbl(data string, delay int,
distance int, origin string, destination string)
using csv options(path "./data/departuredelays.csv")
""")

res17: org.apache.spark.sql.DataFrame = []


In [35]:
val df_sfo = spark.sql("""SELECT date, delay, origin, destination FROM
us_delay_flights_tbl WHERE origin = 'SFO'""")

df_sfo: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 2 more fields]


In [36]:
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_sfo_global_tmp_view")

In [38]:
spark.sql("""
select * from global_temp.us_origin_airport_sfo_global_tmp_view
""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
|1011005|   -8|   SFO|        DFW|
|1011800|    0|   SFO|        ORD|
|1011740|   -7|   SFO|        LAX|
|1012015|   -7|   SFO|        LAX|
|1012110|   -1|   SFO|        MIA|
+-------+-----+------+-----------+
only showing top 10 rows



In [39]:
spark.catalog.dropGlobalTempView("us_origin_airport_sfo_global_tmp_view")

res21: Boolean = true


In [41]:
spark.catalog.listDatabases().show()

+--------------+-------------+----------------+--------------------+
|          name|      catalog|     description|         locationUri|
+--------------+-------------+----------------+--------------------+
|       default|spark_catalog|default database|file:/Users/jiash...|
|learn_spark_db|spark_catalog|                |file:/Users/jiash...|
+--------------+-------------+----------------+--------------------+



In [42]:
spark.catalog.listTables().show()

+--------------------+-------------+----------------+-----------+---------+-----------+
|                name|      catalog|       namespace|description|tableType|isTemporary|
+--------------------+-------------+----------------+-----------+---------+-----------+
|us_delay_flights_tbl|spark_catalog|[learn_spark_db]|       NULL| EXTERNAL|      false|
|us_delay_flights_tbl|         NULL|              []|       NULL|TEMPORARY|       true|
+--------------------+-------------+----------------+-----------+---------+-----------+



In [44]:
spark.catalog.listColumns("us_delay_flights_tbl").show()

+-----------+-----------+--------+--------+-----------+--------+
|       name|description|dataType|nullable|isPartition|isBucket|
+-----------+-----------+--------+--------+-----------+--------+
|       date|       NULL|     int|    true|      false|   false|
|      delay|       NULL|     int|    true|      false|   false|
|   distance|       NULL|     int|    true|      false|   false|
|     origin|       NULL|  string|    true|      false|   false|
|destination|       NULL|  string|    true|      false|   false|
+-----------+-----------+--------+--------+-----------+--------+



In [49]:
val file = """./data/summary-data/parquet/2010-summary.parquet"""

file: String = ./data/summary-data/parquet/2010-summary.parquet


In [50]:
val df = spark.read.format("parquet").load(file)

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [54]:
val json_file = """./data/summary-data/json/*"""

json_file: String = ./data/summary-data/json/*


In [55]:
val df = spark.read.format("json").load(json_file)

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [56]:
val csv_file = """./data/summary-data/csv/*"""

csv_file: String = ./data/summary-data/csv/*


In [57]:
val schema = "dest_country_name string, origin_country_name string, count int"

schema: String = dest_country_name string, origin_country_name string, count int


In [58]:
val df = spark.read.format("csv")
        .schema(schema)
        .option("header", "true")
        .option("mode", "FAILFAST")
        .option("nullValue", "")
        .load(csv_file)

df: org.apache.spark.sql.DataFrame = [dest_country_name: string, origin_country_name: string ... 1 more field]


In [59]:
df.show(10)

+-----------------+-------------------+-----+
|dest_country_name|origin_country_name|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [61]:
val avro_file =  """./data/summary-data/avro/*"""

avro_file: String = ./data/summary-data/avro/*


In [64]:
val orc_file = """./data/summary-data/orc/*"""

orc_file: String = ./data/summary-data/orc/*


In [65]:
val df = spark.read.format("orc")
    .load(orc_file)

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [66]:
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows

